## Lab 2.1

### LAB 2.1.1

In [3]:
import numpy as np
import cv2

# 1. อ่านภาพเข้ามาเป็น Image Array
image = cv2.imread('./img/lab1-img1.jpg', cv2.IMREAD_GRAYSCALE)
height, width = image.shape

# 2. ปรับค่าพิกเซลของภาพโดยใช้สมการเส้นตรง g(x,y) = aƒ(x,y) + b
# และ 3. ปรับค่าพิกเซลที่เกินช่วงค่า bit_dept = 8
def adjust_pixel_value(image, a, b):
    new_image = np.zeros_like(image)
    for i in range(height):
        for j in range(width):
            value = a * image[i, j] + b
            if value > 255:
                value = 255
            elif value < 0:
                value = 0
            new_image[i, j] = value
    return new_image

# สร้างภาพทั้งหมด
images = []
for a in np.linspace(0.5, 1.5, 5):  # ปรับค่า a จาก 0.5 ถึง 1.5 ด้วยจำนวน 5 ค่า
    for b in np.linspace(-50, 50, 4):  # ปรับค่า b จาก -50 ถึง 50 ด้วยจำนวน 4 ค่า
        images.append(adjust_pixel_value(image, a, b))

# 4. บันทึกภาพทั้งหมดเป็นไฟล์วิดีโอ .mp4
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('./output/output_video.mp4', fourcc, 1, (width, height), isColor=False)

for img in images:
    out.write(img)

out.release()
